In [107]:
import os
import pandas as pd
import numpy as np
import time
import random
import urllib
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [109]:
tass = pd.read_excel('tass_excel.xlsx')
comp = tass[pd.notnull(tass.loc[:,'CompanyName'])]
comp['CompName'] = comp['CompanyName'].str.replace('.','').replace(regex=True,to_replace=r'\(.*\)',value=r'').str.strip()
comp_abbr = ['llc','inc','lp','ltd','sa','llp','lc','co','corp','ldc','limited','corperation']
comp['CompNameRep'] = np.where(comp['CompName'].str.rsplit(' ',1).str[1].str.lower().isin(comp_abbr), comp['CompName'].str.rsplit(' ',1).str[0], comp['CompName'])
print('Total number of Companies :',len(comp))

Total number of Companies : 6788


C:\Users\Rocku\Anaconda64\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Rocku\Anaconda64\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [105]:
driver = webdriver.Chrome('chromedriver.exe')

In [106]:
for ind, company in comp.iterrows():
    ex_comp_id = company['CompanyID']
    ex_comp_name = company['CompNameRep']
    driver.switch_to_window(driver.window_handles[0])

    driver.get("http://www.lexisnexis.com/hottopics/lnacademic")

    #Search keyword (submit)
    time.sleep(3)
    driver.switch_to.frame("mainFrame")
    elem = driver.find_element_by_id('terms')
    elem.clear()
    elem.send_keys(ex_comp_name)
    elem.submit()

    time.sleep(3 + (random.random() * 3))
    driver.switch_to_window(driver.window_handles[0])
    driver.switch_to_default_content()
    driver.switch_to_frame('mainFrame')
    dyn_frame = driver.find_element_by_xpath('//frame[contains(@name, "fr_resultsNav")]')
    framename = dyn_frame.get_attribute('name')
    driver.switch_to_frame(framename)
    # Get total number of news
    total_news = int(driver.find_element_by_name('totalDocsInResult').get_attribute('value'))

    if total_news > 500 & total_news <= 3000:
        initial = 1
        final = 500
        batch = 0
        while final <= total_news and final >= initial:
            batch += 1
            time.sleep(3 + (random.random() * 3))
            driver.find_element_by_css_selector('img[alt=\"Download Documents\"]').click()
            driver.switch_to_window(driver.window_handles[1])
            driver.find_element_by_xpath('//select[@id="delFmt"]/option[text()="Text"]').click()
            driver.find_element_by_id('rangetextbox').clear()
            driver.find_element_by_id('rangetextbox').send_keys('{}-{}'.format(initial, final))
            time.sleep(1)
            driver.find_element_by_css_selector('img[alt=\"Download\"]').click()

            try:
                element = WebDriverWait(driver, 120).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, '.TXT')))
                results_url = driver.find_element_by_partial_link_text('.TXT')
                url_href = results_url.get_attribute('href')
                filename = 'dnldFileName=' + '--'.join([str(ex_comp_id),str(ex_comp_name),str(batch)]) + '.txt'
                replaced_href=re.sub(r'dnldFileName.*TXT',filename, url_href, flags=re.DOTALL)
                driver.execute_script("arguments[0].href ='%s';" %replaced_href, results_url)
                results_url.click()
                time.sleep(5)
                driver.close()
            except TimeoutException:
                pass

            initial += 500
            if final + 500 > total_news:
                final = total_news
            else:
                final += 500
            driver.switch_to_window(driver.window_handles[0])
            driver.switch_to_default_content()
            driver.switch_to_frame('mainFrame')
            framelist = driver.find_elements_by_xpath('//frame[contains(@name, "fr_resultsNav")]')
            framename = framelist[0].get_attribute('name')
            driver.switch_to_frame(framename)
    
    elif total_news <= 500:
        time.sleep(3 + (random.random() * 3))
        driver.find_element_by_css_selector('img[alt=\"Download Documents\"]').click()
        driver.switch_to_window(driver.window_handles[1])
        driver.find_element_by_xpath('//select[@id="delFmt"]/option[text()="Text"]').click()
        driver.find_element_by_id('rangetextbox').clear()
        driver.find_element_by_id('rangetextbox').send_keys('{}-{}'.format(initial, final))
        time.sleep(1)
        driver.find_element_by_css_selector('img[alt=\"Download\"]').click()
        
        try:
            element = WebDriverWait(driver, 120).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, '.TXT')))
            results_url = driver.find_element_by_partial_link_text('.TXT')
            url_href = results_url.get_attribute('href')
            filename = 'dnldFileName=' + '--'.join([str(ex_comp_id),str(ex_comp_name)]) + '.txt'
            replaced_href=re.sub(r'dnldFileName.*TXT',filename, url_href, flags=re.DOTALL)
            driver.execute_script("arguments[0].href ='%s';" %replaced_href, results_url)
            results_url.click()
            time.sleep(5)
            driver.close()
        except TimeoutException:
            pass
    
    else:
        pass
        